In [ ]:
import time
import datetime
from collections import OrderedDict
import threading
import random
import collections
import os
import numpy as np
import nidaqmx
import pickle
import sys
import pandas as pd

import quantities as pq
from nidaqmx.stream_writers import (
    DigitalSingleChannelWriter, DigitalMultiChannelWriter)
from nidaqmx.utils import flatten_channel_string
from nidaqmx.constants import (
    LineGrouping, AcquisitionType, DigitalWidthUnits, Edge,
    HandshakeStartCondition, Level, MIOAIConvertTimebaseSource,
    OverflowBehavior, TaskMode, Polarity, RegenerationMode,
    SampleInputDataWhen, SampleTimingType, UnderflowBehavior)
from nidaqmx.error_codes import DAQmxErrors, DAQmxWarnings
from nidaqmx.errors import (
    check_for_error, is_string_buffer_too_small, DaqError, DaqResourceWarning)

from scipy.optimize import fsolve, least_squares
np.set_printoptions(precision=4)

from olfactometer.logical_olfactometer import LogicalOlfactometer
from olfactometer.equipment import AirSupply, YConnector, Mask, Olfactometer
from olfactometer.my_equipment import MyManifold, MyValve, MyJar, MyLowMFC, \
                                      MyMediumMFC, MyHighMFC, PTFETube, StiffTube, \
                                      STUB_LENGTH
from olfactometer.odorants import Solution, Compound, ChemicalOrder, \
                                  Vendor, Molecule
# from graph import make_graph, draw_graph
from pprint import pprint
from olfactometer.smell_engine_communicator import SmellEngineCommunicator
from olfactometer.data_container import DataContainer
from olfactometer.ui import UI
from olfactometer.PID_tester import PID_Tester
from IPython.display import display

In [ ]:
import numpy as np
from tqdm import tqdm
import itertools
import pandas as pd
from scipy.spatial import KDTree
import math

# Generate Concentration Space 

In [ ]:
NUM_DIMENSIONS = 1 #Value of 
import sys

conc_set = [0]+np.geomspace(1e-9, 1e-3, num=49).tolist()   # Generate evenly distributed logarithmic list of concentration values ranging from [0.01, 1]

multi_dim_conc = list(itertools.product(conc_set, repeat=NUM_DIMENSIONS))    

def closest_point(conc) -> (float,int):
    rounded_points = np.ones(len(conc))
    points = np.ones(len(conc))
    for idx, axis in enumerate(conc): 
        points[idx] = axis
        rounded_points[idx]=min(conc_set, key=lambda xP:abs(xP-axis))
    dist = np.sum(np.square(points - rounded_points))
    index = multi_dim_conc.index(tuple(rounded_points))
    return dist, index


# Initialize Code To Generate Hardware States

In [ ]:
NUM_VALVES = 10
NUM_BLANK_JARS = 2
NUM_ODORANT_JARS = 1 #Ethanol
NUM_MFC_SETPOINTS = 15
# Generate evenly distributed logarithmic list of concentration values ranging from [0.01, 1]
JAR_CONCENTRATIONS = np.geomspace(0.01, 1, num=7).tolist() + (np.linspace(0,1,num=10)).tolist()[0:9] #[0]+np.geomspace(0.01, 1, num=7).tolist()

# Read in valve states and number of active jars with odorants
def valve_combinations(possible_valve_states):
    combos = []
    empty_jars = [0.0 for i in range(0,(NUM_VALVES - (NUM_ODORANT_JARS+NUM_BLANK_JARS)))] # Generate list of empty jars, +1 for empty jar
    for x in possible_valve_states:    # For A states
        for y in possible_valve_states:    # For B states
            if (x == 0 or y ==0):     # ensure no valve is in both state A && B
                combos.append((float(x),float(y)))
    # Cartesian product of length number of jars with odorants +1 (for empty jar balance) => combination of all possible individual valve states within range                     
    valve_states = list(itertools.product(combos, repeat = NUM_ODORANT_JARS + NUM_BLANK_JARS))    
    return_state = []
    return_states = []
    # Convert tuple pairs (state A, state B) into list of wA[0:10] union wB[0:10]
    for state in valve_states: 
        a_s = []
        b_s = []
        for (a,b) in state:             
            #return_state = return_state + [a,b]
            a_s.append(float(a))
            b_s.append(float(b))
        return_state = a_s + empty_jars + b_s + empty_jars  # Pad occupied jars in respective state with remaining (empty) jars in respective state
        if (1 in a_s and 1 in b_s):    # Ensure there is at least 1 jar with full unit of occupancy per smell frame
            return_states.append(return_state)
        return_state = []
    return(return_states)

# Generate list of possible MFC setpoints union valve states
# param: valve_combos is the returned result of valve_combinations()
def mfc_combinations(start,end, num, valve_combos):
    total_combos = []
    if(start == 0): start = 0.01
    if(end == 0): end = 0.01
    mfc_combs = [0]+np.geomspace(start, end, num=num).tolist()   # evenly distributed logarithmic list of mfc setpoints ranging from [0.01, 1]
    # For returned valve states, append combination of MFC setpoints
    for state in valve_combos:
        for A in mfc_combs:     # For mfc A values
            for B in mfc_combs:  # for mfc B values
                if (B > 0.1): # if operating above 1 cc/min
                    total_combos.append([float(A),float(B)] + state)
    return(total_combos)

In [ ]:
mfc_combs = [0]+np.geomspace(0.01, 1, num=15).tolist()
print(mfc_combs)

# Generate Hardware States
The number of possible states to check is very large.

In [ ]:
# olfactometer_combinations = valve_combinations(JAR_CONCENTRATIONS)
olfactometer_combinations = mfc_combinations(0,1,NUM_MFC_SETPOINTS,valve_combinations(JAR_CONCENTRATIONS))

print(len(olfactometer_combinations))
# print(olfactometer_combinations[10])


# Import Olfactometer Dependencies 

# Initialize Molecules and Smell Engine

In [ ]:
# Initialize # molecules
molecules = OrderedDict([(702, 'Ethanol')])

In [ ]:
# NUM_PID_SAMPLES = 50
smell_engine = SmellEngine(1000, DataContainer(), debug_mode=True, write_flag=False, PID_mode = False, oms=molecules)  # WRITE MODE ENABLED
# Initialize system
smell_engine.set_odorant_molecule_ids(list(molecules.keys()))
smell_engine.set_odorant_molecule_dilutions([1])
smell_engine.initialize_smell_engine_system()
smell_engine.olf.loaded_molecules

# Generate Data
Note this is the cell that takes a long time to execute. 

In [ ]:
# Format data for generating Pandas Dataframe
max_flow_rates = smell_engine.olf.get_max_flow_rates()
total_vapor = smell_engine.olf.get_vapor_concs_dense(set(list(smell_engine.target_concentration)))
A = smell_engine.olf.A.magnitude
n_jars = len(smell_engine.olf.jars)
NUMBER_OF_JARS_WITH_ODORANTS = 1

#alpha = 1e-25

NUMBER_THREADS = 6
threading_results = [1]*NUMBER_THREADS

def generate_optimal_concentration(olfactometer_combinations,index):
    length = len(olfactometer_combinations)
    optimal_concentration_configurations = [(float("inf"),float("inf"))]*len(multi_dim_conc)
    for i in range(length):
        if(i %int(length/1000) == 0): print(f"Thread {index} {(i/length)*1000}") #Progress "bar" old one breaks with threading
        new_conc_combo = list(smell_engine.olf.calc_conc(np.array(olfactometer_combinations[i]), n_jars, max_flow_rates, A).sum(axis=0)[:NUMBER_OF_JARS_WITH_ODORANTS])
        _dist, _index = closest_point(new_conc_combo)
        #Penalty Code based on alpha
        valve_states = np.array(olfactometer_combinations[i][2:])
#         penalty = alpha * np.nan_to_num(np.reciprocal(valve_states),posinf=0, neginf=0).sum() 
        #print(olfactometer_combinations[i][2:],np.nan_to_num(np.reciprocal(valve_states),posinf=0, neginf=0))
        #Penalty Code End 
        if(optimal_concentration_configurations[_index][0]>_dist):
            optimal_concentration_configurations[_index] = (_dist, olfactometer_combinations[i])
    print(f"Thread {index} is ending")
    threading_results[index] = optimal_concentration_configurations


In [ ]:
import logging
import threading


#split olfactometer_combinations by number of threads 
olfactometer_combinations_list = []
combination_list_length = len(olfactometer_combinations) / NUMBER_THREADS

for i in range(0,NUMBER_THREADS):
    olfactometer_combinations_list.append(olfactometer_combinations[int(i*combination_list_length):int((i+1)*combination_list_length)])
    
threads = []
for i in range(0,NUMBER_THREADS):
    _thread = threading.Thread(target = generate_optimal_concentration,args = (olfactometer_combinations_list[i],i,))
    threads.append(_thread)
    _thread.start()

In [ ]:

final_optimal_concentration_configurations = threading_results[0]


for results in threading_results[1:]:
    for idx, (dist,config) in enumerate(results): 
        if(final_optimal_concentration_configurations[idx][0]>dist):
            final_optimal_concentration_configurations[idx]=(dist,config)
    
optimal_concentration_configurations = final_optimal_concentration_configurations #Works with remaining code


# Format for Data

In [ ]:
body_values = []
index_values = []

for idx, machine_config in enumerate(optimal_concentration_configurations):
    if (machine_config[0] != float("inf")):   # Safety check for impossible configs
        index_values.append(tuple(machine_config[1])) # Store machine configuration as index for dataframe
        body_values.append(multi_dim_conc[idx]) # Store concentration as body emtry for dataframe

In [ ]:
print(f"There are {len(multi_dim_conc)-len(body_values)} unobtainnable results.")

# Wrtie to Data Frame and then Save

In [ ]:
# Generate dataframe indexed by oflactometer configurations and body containing concentration values
df = pd.DataFrame(body_values, index=index_values, columns=['Ethanol'])

In [ ]:
df.to_pickle("./ethanol_11-2-21.pkl")

# Data Table Validation

In [ ]:
# Data Table to Read In
df = pd.read_pickle("./ethanol_11-2-21.pkl")

In [ ]:
state_array = []
for idx, state in enumerate(df.index):
    if(list(state)[-1] != 0):
        print(list(state)[-1])
    conc = [df["Ethanol"][idx]]
    state_array.append(conc + list(state))

In [ ]:
df_states = pd.DataFrame(state_array, columns=["cE","mfcA","mfcB","h1","h2","h3","h4","h5","h6","h7","h8","h9","h10","L1","L2","L3","L4","L5","L6","L7","L8","L9","L10"])

In [ ]:
df_states

In [ ]:
df_states[12:13]

In [ ]:
# df_states.loc[(df_states["mfcB"]>0.2) & (df_states["cA"] == 0) & (df_states["dL"] == 0)]
df_states.loc[(df_states["cC"]>0)] #df_states["cE"]== 0)]